<a href="https://colab.research.google.com/github/vuduclyunitn/learning_python/blob/master/T%C3%ACm_hi%E1%BB%83u_decoators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Python decorators là một cách hữu hiệu để thay đổi các hàm (functions). Decorators được giới thiệu lần đầu tiên ở phiên bản Python 2.2, với các decorators đầu tiên  ```classmethod()``` và ```staticmethod()```, sau này các decorators được mở rộng giúp cho chúng trở nên mềm dẻo và dễ đọc. Cùng với hai decorators ban đầu, hiện tại Python cung cấp một vài decorators khác và cho phép ta tạo các decorators tuỳ chỉnh. 
Tuy nhiên dường như hầu hết các developers không hiểu cách chúng hoạt động thực sự. 

Bài viết này mong muốn thay đổi điều đó - chúng tôi sẽ trình bài khái niệm decorator và cách sử dụng nó, cũng như là cách tạo các decorators của riêng bạn. Sau đó chúng ta sẽ xem cách sử dụng các decorators để tạo các phương thức tĩnh, lớp và các phương thức trừu tượng và tìm hiểu hàm ```super()```, hàm cho phép ta đặt đoạn mã có thể triển khai được (implementable code) bên trong một phương thức trừu tượng.



## Decorators là gì và khi nào sử dụng chúng

Một decorator là một hàm mà nhận một hàm khác như là một tham số và thay thế hàm này với một hàm mới đã được sửa đổi. Ta sử dụng decorator khi muốn bao các đoạn mã phổ biến khi chúng ta gọi chúng trước, sau, và xung quanh nhiều hàm. Nếu bạn đã từng viết mã Emacs Lisp, bạn có thể đã sử dụng decorator ```defadvice``` để cho phép bạn định nghĩa mã được gọi xung quanh một hàm. Nếu bạn đã sử dụng các kết hợp phương thức trong Common Lisp Object System (CLOS) thì Python decorators có các khái niệm tương tự. 
Chúng ta sẽ xem các định nghĩa đơn giản của decorator, và sau đó chúng ta sẽ phân tích các hoàn cảnh phổ biến ta cần sử dụng chúng

### Tạo ra các Decorators
Khả năng cao là bạn đã sử dụng các decorators để tạo ra các hàm bao của riêng bạn. Decorator đơn giản nhất đó là hàm ```identity```, hàm này không làm gì ngoài việc trả về hàm ban đầu. Dưới đây là định nghĩa của nó

In [0]:
def identity(f):
  return f

Bạn có thể sử dụng decorator trên như sau:

In [0]:
@identity
def foo():
  return 'bar'

Bạn nhập tên của decorator sau kí tự @ và sau đó viết hàm bạn muốn sử dụng decorator. Cách viết trên giống với đoạn mã sau:

In [0]:
def foo():
  return 'bar'
foo = identity(foo)

Decorator trên không đem lại ích lợi gì, nhưng nó chạy được. Hãy nhìn vào một ví dụ khác

In [0]:
_functions = {}

def register(f):
  global _functions
  _functions[f.__name__] = f
  return f


In [0]:
@register
def foo():
  return 'bar'

In [0]:
@register
def bar():
  return 'foo'

In [0]:
foo()

'bar'

In [0]:
_functions['foo']

<function __main__.foo>

In [0]:
bar()

'foo'

In [0]:
_functions['bar']

<function __main__.bar>

Ở ví dụ trên ta sử dụng ```register``` decorator để lưu tên hàm được decorated vào một từ điển. Biến từ điển ```_functions``` có thể được sử dụng sau đó để truy cập tới tên các hàm đã được gọi thông qua cú pháp ```_functions[function name]``` ví dụ ```_functions['foo']``` trỏ tới hàm ```foo()```

Trong phần sau, tôi sẽ giải thích cách viết các decorators cho riêng bạn. Sau đó tôi sẽ trình bày cách các decorators có sẵn được cung cấp bởi Python và giải thích cách (và khi nào) sử dụng chúng

### Viết các decorators
Như đã nhắc tới trước đó, decorators thường được sử dụng khi refactoring các mã lặp lại xung quanh các hàm. Xem xét tập hợp các hàm kiểm tra tên người dùng nhập vào có phải là người quản trị hay khôgn, nếu người dùng không phải là quản trị viên, đưa ra một ngoại lệ.

In [0]:
class Store(object):
  def get_food(self, username, food):
    if username != 'admin':
      raise Exception("This user is not allowed to get food")
    return self.storage.get(food)

  def put_food(self, username, food):
    if username != 'admin':
      raise Exception("This user is not allowed to put food")
    self.storage.put(food)

Bạn có thể thấy một vài đoạn mã bị lặp lại ở đây. Bước đầu tiên để làm cho đoạn mã này hiệu quả hơn đó là factor được mã kiểm tra trạng thái người quản trị.

In [0]:
def check_is_admin(username):
  if username != 'admin':
    raise Exception("This user is not allowed to get or put food")

class Storage(object):
  def get_good(self, username, food):
    check_is_admin(username)
    return self.storage.get(food)
  
  def put_food(self, username, food):
    check_is_admin(username)
    self.storage.put(food)

Chúng ta đã chuyển hàm kiểm tra người quản trị thành một hàm riêng. Đoạn mã của chúng ta bây giờ nhìn gọn hơn, nhưng chúng ta vẫn có thể làm tốt hơn nếu dùng một decorator

In [0]:
def check_is_admin(f):
  def wrapper(*args, **kwargs):
    if kwargs.get('username') != 'admin':
      raise Exception("This user is not allowed to get or put food")
    return f(*args, **kwargs)
  return wrapper


In [0]:
class Store(object):
  @check_is_admin
  def get_food(self, username, food):
    return self.storage.get(food)
  
  @check_is_admin
  def put_food(self, username, food):
    self.storage.put(food)
    

Chúng ta định nghĩa một decorator ```check_is_admin``` và sau đó gọi nó bất cứ khi nào ta cần kiểm tra các quyền truy cập. Decorator này xem xét các tham số gửi vào hàm thông qua biến ```kwargs``` và lấy ra giá trị truyền vào ```username```, thông hiện việc kiểm tra trước khi gọi hàm thực sự. Sử dụng các decorators như trên giúp cho việc quản lý các hàm sử dụng chung trở nên dễ dàng hơn. Đối với những người có nhiều kinh ngiệm lập trình Python, điều này không có gì mới, nhưng điều bạn có thể không nhận ra là cách tiếp cận ngây thơ trên (hiện thực các decorators) có nhiều yếu điểm chính.

### Chồng lên các Decorators
Bạn có thể sử dụng một vài decorators cho một hàm hoặc phương thức

In [0]:
def check_user_is_not(username):
  def user_check_decorator(f):
    def wrapper(*args, **kwargs):
      if kwargs.get('username') == username:
        raise Exception("This user is not allowed to get food")
      return f(*args, **kwargs)
    return wrapper
  return user_check_decorator

  

In [0]:
class Store(object):
  @check_user_is_not("admin")
  @check_user_is_not("user123")

  def get_food(self, username, food):
    return self.storage.get(foodhehe)

Ở ví dụ trên ```check_user_is_not()``` là một hàm factory cho decorator của chúng ta ```user_check_decorator()```. Nó tạo ra một hàm decorator phụ thuộc vào biến ```username``` và trả về biến đó. Hàm ```user_check_decorator()``` sẽ phục vụ như là một hàm decorator cho ```get_food()```. 

Hàm ```get_food()``` được decorated hai lần sử dụng ```check_user_is_not()```. Câu hỏi ở đây là username nào được kiểm tra trước - ```admin``` hay là ```user123```. Câu trả lời nằm ở đoạn code dưới đây, tôi chuyển đoạn mã phía trên thành đoạn code tương ứng mà không dùng decorator

In [0]:
class Store(object):
  def get_food(self, username, food):
    return self.storage.get(food)

Store.get_food = check_user_is_not("user123")(Store.get_food)
Store.get_food = check_user_is_not("admin")(Store.get_food)

Danh sách decorator được áp dụng từ đỉnh tới đáy, vì vậy các decorators gần với keyword ```def``` nhất sẽ được áp dụng đầu tiên và thực thi cuối. Trong ví dụ trên, chương trình sẽ kiểm tra ```admin``` trước và sau đó là ```user123```

### Viết các Class Decorators
Ta cũng có thể viết các class decorators, mặc dù việc này ít được thực hiện trong thực tế. *Class decorators* hoạt động giống với function decorators, nhưng chúng được sử dụng với class thay vì các hàm. Ví dụ dưới đây là một class decorator thiết lập các thuộc tính cho hai lớp:

In [0]:
import uuid

def set_class_name_and_id(klass):
  klass.name = str(klass)
  klass.random_id = uuid.uuid4()
  return klass

@set_class_name_and_id
class SomeClass(object):
  pass

Khi class được nạp và định nghĩa, nó sẽ thiết lập các thuộc tính, như sau

In [29]:
SomeClass.name

"<class '__main__.SomeClass'>"

In [30]:
SomeClass.random_id

UUID('e0f5c75a-89e5-4bfc-9a24-d1fde37343f9')

Như đối với các function decorators, sẽ có thể hữu ích nếu ta có thể bao các đoạn code chung chỉnh sửa các lớp.

Ta cũng có thể các class decorators để bao một hàm hoặc lớp với các lớp khác. Ví dụ, class decorators thường được dùng được sử dụng để bao một hàm đang lưu một trạng thái. Ví dụ sau bao một hàm print() để kiểm tra xem nó được gọi bao nhiều 

In [0]:
class CountCalls(object):
  def __init__(self, f):
    self.f = f
    self.called = 0

  def __call__(self, *args, **kwargs):
    self.called += 1
    return self.f(*args, **kwargs)

In [0]:
@CountCalls
def print_hello():
  print("hello")

In [33]:
print_hello.called

0

In [34]:
print_hello()

hello


In [35]:
print_hello.called

1

In [36]:
print_hello()

hello


In [37]:
print_hello.called

2

### Thu thập các thuộc tính ban đầu với update_wrapper Decorator

Như đã nhắc tới ở phần trước, một decorator thay thế hàm ban đầu với một hàm mới một cách nhanh chóng. Tuy nhiên, hàm mới thiếu đi rất nhiều thuộc tính của hàm ban đầu, ví dụ như docstring và tên của hàm. Đoạn code dưới đây cho ta thấy cách hàm ```foobar()``` mất đi thuộc tính docstring và tên của nó mỗi khi nó được decorated với ```is_admin``` decorator. 

In [0]:
def is_admin(f):
  def wrapper(*args, **kwargs):
    if kwargs.get('username') != 'admin':
      raise Exception("This user is not allowed to get food")
    return f(*args, **kwargs)
  return wrapper

In [0]:
def foobar(username="someone"):
  """Do crazy stuff."""
  pass

In [44]:
foobar.__doc__

'Do crazy stuff.'

In [41]:
foobar.__name__

'foobar'

In [0]:
@is_admin
def foobar(username="someone"):
  """Do crazy stuff."""
  pass

In [0]:
foobar.__doc__

In [47]:
foobar.__name__

'wrapper'

Không có thuộc tính docstring và tên hàm đúng có thể là một vấn đề trong nhiều hoàn cảnh, ví dụ như sinh ra tài liệu cho mã nguồn

May mắn thay, module ```functools``` trong thư viện Python chuẩn giải quyết vấn đề này với hàm ```update_wrapper()``` cho phép sao chép các thuộc tính từ hàm ban đầu nhưng bị mất định bởi hàm bao. 

In [48]:
import functools

def foobar(username="someone"):
  """Do crazy stuff"""
  pass

foobar = functools.update_wrapper(is_admin, foobar)
foobar.__name__

'foobar'

In [49]:
foobar.__doc__

'Do crazy stuff'

Bay giờ thì hàm ```foobar()``` có tên và docstring đúng thậm chí khi nó được decorated với ```is_admin```.

### Trích xuất thông tin liên quan với inspect

Trong các ví dụ ta đã thấy qua ta giả định rằng hàm bị decorated sẽ luôn luôn có ```username``` được truyền vào nó thông qua tham số từ khoá (keyword argument), nhưng có nhiều trường hợp khác. Ta có thể nhận được nhiều thông tin mà ta dùng để trích xuất username. Với suy nghĩ đó, ta sẽ xây dựng một phiên bản thông minh hơn giúp ta nhìn vào các tham số của các hàm decorated và lấy ra những thứ ta cần.

Để làm được điều đó, Python sử dụng module ```inspect```, module cho phép ta lấy ra chữ kí của hàm và xử lý nó.

In [0]:
import functools
import inspect

def check_is_admin(f):
  @functools.wraps(f)
  def wrapper(*args, **kwargs):
    func_args = inspect.getcallargs(f, *args, **kwargs)
    if func_args.get('username') != 'admin':
      raise Exception("This user is not allowed to get food")
    return f(*args, **kwargs)
  return wrapper

@check_is_admin
def get_food(username, type='chocolate'):
  return type + " nom nom nom!"

In [55]:
get_food("admin")

chocolate


'chocolate nom nom nom!'

Hàm trên gọi ```inspect.getcallargs()``` nhận về một từ điển chứa tên và và các giá trị của các tham số như là các cặp key-value. Trong ví dụ của chúng ta, hàm này trả về ```{'username': 'admin', 'type': 'chocolate'}```. Có nghĩa rằng decorator của chúng ta không phải kiểm tra xem tham số ```username``` có phải là argument theo vị trí hay keyword hay không; tất cả những gì ta cần làm là tìm kiếm ```username``` trong từ điển của chúng ta. 

Sử dụng ```functools.wraps``` và module ```inspect```, bạn có thể viết bất cứ decorator tuỳ chỉnh nào mà bạn cần. Tuy nhiên, đừng lạm dụng module ```inspect``` quá: trong khi nó có thể đoán những thứ một hàm sẽ chấp nhận như là một argument, khả năng của ```inspect``` có thể mong manh, dễ vỡ khi chữ kí của hàm thay đổi. Decorators là một cách tuyệt vời để triển khai câu thần chú *Don't Repeat Yourself* vì vậy nó được yêu thích bởi các developers.